In [2]:
import pygame
import random
import time

pygame.init()

# Настройки экрана 
WIDTH, HEIGHT = 600, 400
BLOCK_SIZE = 20
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game with Food Timer & Weights")

# Цвета 
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (200, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)  # Еда с другим весом

#  Шрифты 
font = pygame.font.SysFont("Arial", 24)
big_font = pygame.font.SysFont("Arial", 36)

# --- Таймер ---
clock = pygame.time.Clock()

# Настройки еды 
FOOD_LIFETIME = 5000  # миллисекунды (5 секунд)

# Генерация границ 
def generate_walls():
    walls = []
    for x in range(0, WIDTH, BLOCK_SIZE):
        walls.append((x, 0))  # верх
        walls.append((x, HEIGHT - BLOCK_SIZE))  # низ
    for y in range(0, HEIGHT, BLOCK_SIZE):
        walls.append((0, y))  # левая
        walls.append((WIDTH - BLOCK_SIZE, y))  # правая
    return walls

# Генерация новой еды с весом и временем создания 
def generate_food(snake, walls):
    while True:
        x = random.randint(1, (WIDTH // BLOCK_SIZE) - 2) * BLOCK_SIZE
        y = random.randint(1, (HEIGHT // BLOCK_SIZE) - 2) * BLOCK_SIZE
        pos = (x, y)
        if pos not in snake and pos not in walls:
            weight = random.choice([1, 2, 3])  # вес еды (очки)
            color = RED if weight == 1 else YELLOW  # разные цвета для веса
            timestamp = pygame.time.get_ticks()  # время появления еды
            return {'pos': pos, 'weight': weight, 'color': color, 'time': timestamp}

# Основная функция игры 
def run_game():
    snake = [(100, 100)]  # стартовая позиция змейки
    direction = (BLOCK_SIZE, 0)  # начальное направление — вправо
    walls = generate_walls()
    food = generate_food(snake, walls)
    score = 0
    level = 1
    speed = 10
    game_over = False

    while True:
        clock.tick(speed)  # частота кадров зависит от скорости

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False  # выйти из игры
            if event.type == pygame.MOUSEBUTTONDOWN and game_over:
                if event.button == 1:
                    return True  # клик — перезапуск
            if event.type == pygame.KEYDOWN and not game_over:
                if event.key == pygame.K_UP and direction != (0, BLOCK_SIZE):
                    direction = (0, -BLOCK_SIZE)
                elif event.key == pygame.K_DOWN and direction != (0, -BLOCK_SIZE):
                    direction = (0, BLOCK_SIZE)
                elif event.key == pygame.K_LEFT and direction != (BLOCK_SIZE, 0):
                    direction = (-BLOCK_SIZE, 0)
                elif event.key == pygame.K_RIGHT and direction != (-BLOCK_SIZE, 0):
                    direction = (BLOCK_SIZE, 0)

        if not game_over:
            # Движение змеи
            head = (snake[0][0] + direction[0], snake[0][1] + direction[1])
            snake.insert(0, head)

            # Проверка на проигрыш
            if (
                head in snake[1:] or
                head in walls or
                head[0] < 0 or head[0] >= WIDTH or
                head[1] < 0 or head[1] >= HEIGHT
            ):
                game_over = True
            elif head == food['pos']:
                score += food['weight']  # учёт веса еды
                if score % 4 == 0:
                    level += 1
                    speed += 2  # повышение уровня = увеличение скорости
                food = generate_food(snake, walls)  # новое появление еды
            else:
                snake.pop()  # если еду не съели — хвост удаляется

            # Проверка времени жизни еды
            if pygame.time.get_ticks() - food['time'] > FOOD_LIFETIME:
                food = generate_food(snake, walls)  # еда исчезла — новая

        #  Отрисовка экрана 
        screen.fill(BLACK)

        # Стены
        for wall in walls:
            pygame.draw.rect(screen, BLUE, (*wall, BLOCK_SIZE, BLOCK_SIZE))

        # Змейка
        for part in snake:
            pygame.draw.rect(screen, GREEN, (*part, BLOCK_SIZE, BLOCK_SIZE))

        # Еда (цвет зависит от веса)
        pygame.draw.rect(screen, food['color'], (*food['pos'], BLOCK_SIZE, BLOCK_SIZE))

        # Счёт и уровень
        score_text = font.render(f"Score: {score}  Level: {level}", True, WHITE)
        screen.blit(score_text, (10, 10))

        # Сообщение при проигрыше
        if game_over:
            msg = big_font.render("Game Over!", True, WHITE)
            restart_msg = font.render("Click to Restart or Close", True, WHITE)
            screen.blit(msg, (WIDTH // 2 - msg.get_width() // 2, HEIGHT // 2 - 40))
            screen.blit(restart_msg, (WIDTH // 2 - restart_msg.get_width() // 2, HEIGHT // 2 + 10))

        pygame.display.flip()

#  Главный цикл приложения
while True:
    restart = run_game()
    if not restart:
        break

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
